In [1]:
include("TDCM_full_transition.jl")
include("data_generation.jl")
using Distributions

In [2]:
N, J, K, O, L, S = 5000, 24, 3, 3, 8, 50
num_features_gamma = 3
Q = [0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1;
     0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1;
     0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1]

X = Vector{Vector{Matrix{Float64}}}(undef, K)
for k in 1:K
     X[k] = Vector{Matrix{Float64}}(undef, O)
     X[k][1] = ones(N, 1)
     for t in 2:O
          X[k][t] = Matrix{Float64}(undef, N, 3)
          X[k][t][:,1] = ones(N)
          X[k][t][:,2] = rand(Normal(0, 1), N)
          X[k][t][:,3] = rand(Bernoulli(0.5), N)
     end
end
U = Vector{Vector{Matrix{Float64}}}(undef, K)
for k in 1:K
     U[k] = Vector{Matrix{Float64}}(undef, O)
     for t in 1:O
          U[k][t] = Matrix{Float64}(undef, S, 3)
          U[k][t][:,1] = ones(S)
          U[k][t][:,2] = rand(Bernoulli(0.5), S)
          U[k][t][:,3] = rand(Bernoulli(0.5), S)
     end
end

In [3]:
data = data_generation(Q, X, U, Int(O), Int(K), Int(S), Int(N/S), beta_interact=true);

In [4]:
obs = TDCMObs(data.Y, data.Q, data.U, data.X, data.group);

In [5]:
N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)

mu_beta_prior = Vector{Vector{Float64}}(undef, J)
L_beta_prior = Vector{Matrix{Float64}}(undef, J)
for j in 1:J
    num_features = size(obs.D[j], 2)
    mu_beta_prior[j] = zeros(num_features)
    L_beta_prior[j] = Matrix(1.0I, num_features, num_features)
end

mu_omega_prior = Vector{Vector{Vector{Vector{Vector{Float64}}}}}(undef, K)
L_omega_prior = Vector{Vector{Vector{Vector{Matrix{Float64}}}}}(undef, K)
a_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)
b_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)

for k in 1:K
    mu_omega_prior[k] = Vector{Vector{Vector{Vector{Float64}}}}(undef, O)
    L_omega_prior[k] = Vector{Vector{Vector{Matrix{Float64}}}}(undef, O)
    a_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    b_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    for t in 1:O
        num_features_gamma = size(obs.X[k][t], 2)
        num_features_omega = size(obs.U[k][t], 2)
        if t == 1
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 1)
            mu_omega_prior[k][t][1] = Vector{Vector{Float64}}(undef, 1)
            mu_omega_prior[k][t][1][1] = zeros(num_features_omega)

            L_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 1)
            L_omega_prior[k][t][1] = Vector{Matrix{Float64}}(undef, 1)
            L_omega_prior[k][t][1][1] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)

            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            a_tau_prior[k][t][1] = [.01]

            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            b_tau_prior[k][t][1] = [.01]
        else
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 2)
            L_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 2)
            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            for z in 1:2
                mu_omega_prior[k][t][z] = Vector{Vector{Float64}}(undef, num_features_gamma)
                L_omega_prior[k][t][z] = Vector{Matrix{Float64}}(undef, num_features_gamma)
                a_tau_prior[k][t][z] = ones(num_features_gamma) .* .01
                b_tau_prior[k][t][z] = ones(num_features_gamma) .* .01
                for m in 1:num_features_gamma
                    mu_omega_prior[k][t][z][m] = zeros(num_features_omega)
                    L_omega_prior[k][t][z][m] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)
                end
            end
        end
    end
end

In [6]:
M = 10
model = TDCModel(obs, mu_beta_prior, L_beta_prior, mu_omega_prior, L_omega_prior, a_tau_prior, b_tau_prior, M, enable_parallel=true)
optim = ADAMparams(model);

TDCModel constructed for computation on 16 threads


In [7]:
#Initialize betas in the correct direction
for j in 1:J
    model.mu_beta_star[j] = ones(length(model.mu_beta_star[j]))
    model.mu_beta_star[j][1] = -1
end

# Sample Z values
for i in 1:N
    for t in 1:O
        sample_Z(model, i, t)
    end
end

# Sample beta
sample_β(model)

# Sample gammas
for k in 1:K
    for t in 1:O
        if t == 1
            for s in 1:S
                sample_γ(model, s, t, k, 1)
            end
        else
            for z in 1:(2^(t - 1))
                for s in 1:S
                    sample_γ(model, s, t, k, z)
                end
            end
        end
    end
end

# Sample omega and tau values
for k in 1:K
    for t in 1:O
        num_features = length(model.gamma_sample[k][t][1][1][1])
        if t == 1
            for feature in 1:num_features
                sample_ω(model, k, t, 1, feature)
                sample_τ(model, k, t, 1, feature)
            end
        else
            for z in 1:(2^(t - 1))
                for feature in 1:num_features
                    sample_ω(model, k, t, z, feature)
                    sample_τ(model, k, t, z, feature)
                end
            end
        end
    end
end

In [8]:
for iter in 1:1
    # update_categorical_variational_distribution(model, 1, step=.01, maxiter=5, verbose=false)
    # println("success 1")
    # update_categorical_variational_distribution(model, 2, step=.01, maxiter=5, verbose=false)
    # println("success 2")
    # update_normal_variational_distribution(model, init_step = .01, maxiter = 5, clip = 50.0, verbose = false, ADAMparams=optim)
    # println("success 3")
    # update_normal_variational_distribution2(model, init_step = .01, maxiter = 10, clip = 50.0, verbose = false, ADAMparams=optim)
    # println("success 4")
    # update_normal_variational_distribution3(model, init_step = .01, maxiter = 10, clip = 50.0, verbose = false, ADAMparams=optim)
    # println("success 5")
    update_inverse_gamma_distribution(model, step = .01, maxiter = 5, verbose = false, ADAMparams=optim)
    println("success 6")
end

: 

In [85]:
model.gamma_sample[1][2]

2-element Vector{Vector{Vector{Vector{Float64}}}}:
 [[[0.0313189185199595, -2.573026731585608, 0.40915537348709624], [-0.3355815238051418, -0.5009143891903525, -0.3733144811657611], [0.982667880066323, 0.17198742931086455, 1.0658590638588925], [-0.10544549462831387, 0.5519306760461558, -1.2638657915603728], [0.42815718977696765, -1.198486506941467, 0.8157772472964926], [-1.2153246922444552, 1.0452890965981236, -0.8508438139191158], [1.7836859604209978, 1.236236621930997, 0.796797002195092], [-0.010780317071867293, -0.16400109272910302, -0.3931432118648581], [-0.8670730314873211, 0.393185759826961, -1.6570841148456668], [-2.7599975974820663, 0.022614283975846552, 0.9175840667155665]], [[0.5867789077783991, -0.8894459423509425, 1.018573193583893], [-0.38241262231340906, -0.8415752293820129, -0.15712082487859355], [-1.616317462337529, -0.2716993815679507, -0.9142074753528661], [-0.8720331730404899, 2.6661414695439913, -0.4497847941208155], [2.643677155241491, 0.6594172303702377, -1.615395